In [1]:
from time import sleep
from random import randint

In [2]:
from time import time

In [3]:
start_time = time()

In [4]:
requests = 0

In [5]:
for _ in range(5):
# A request would go here
    requests += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))

Request: 1; Frequency: 0.222186918335 requests/s
Request: 2; Frequency: 0.307529794922 requests/s
Request: 3; Frequency: 0.399726431835 requests/s
Request: 4; Frequency: 0.42078700679 requests/s
Request: 5; Frequency: 0.399693198146 requests/s


In [6]:
from IPython.core.display import clear_output


In [8]:
for _ in range(5):
# A request would go here
    requests += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 15; Frequency: 0.356688037333 requests/s


In [9]:
from requests import get
url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'
response = get(url)
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')

In [10]:
# Redeclaring the lists to store data in
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2018)]
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
headers = {"Accept-Language": "en-US, en;q=0.5"}
# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url +
        '&sort=num_votes,desc&page=' + page, headers = headers)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 72:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request:72; Frequency: 0.0621217394793 requests/s


In [25]:
import pandas as pd

In [13]:
movie_ratings = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes
})
print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3280 entries, 0 to 3279
Data columns (total 5 columns):
imdb         3280 non-null float64
metascore    3280 non-null int64
movie        3280 non-null object
votes        3280 non-null int64
year         3280 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 128.2+ KB
None


,imdb,metascore,movie,votes,year
0,8.5,67,Gladiator,1217972,(2000)
1,8.5,80,Memento,1034713,(2000)
2,8.3,55,Snatch,721743,(2000)
3,8.3,68,Requiem for a Dream,702989,(2000)
4,7.4,64,X-Men,538572,(2000)
5,7.8,73,Cast Away,474561,(2000)
6,7.6,64,American Psycho,433137,(2000)
7,7.3,62,Unbreakable,354116,(2000)
8,7.0,73,Meet the Parents,293713,(2000)
9,6.1,59,Mission: Impossible II,291843,(2000)


In [14]:
movie_ratings['year'] = movie_ratings['year'].str[-5:-1].astype(int)

In [15]:
movie_ratings.head()

,imdb,metascore,movie,votes,year
0,8.5,67,Gladiator,1217972,2000
1,8.5,80,Memento,1034713,2000
2,8.3,55,Snatch,721743,2000
3,8.3,68,Requiem for a Dream,702989,2000
4,7.4,64,X-Men,538572,2000


In [21]:
movie_ratings

,imdb,metascore,movie,votes,year
0,8.5,67,Gladiator,1217972,2000
1,8.5,80,Memento,1034713,2000
2,8.3,55,Snatch,721743,2000
3,8.3,68,Requiem for a Dream,702989,2000
4,7.4,64,X-Men,538572,2000
5,7.8,73,Cast Away,474561,2000
6,7.6,64,American Psycho,433137,2000
7,7.3,62,Unbreakable,354116,2000
8,7.0,73,Meet the Parents,293713,2000
9,6.1,59,Mission: Impossible II,291843,2000
